# Fraud Detection with Variational Autoencoder

- VAE는 vanilla 오토인코더에 비해 훨씬 나은 성능의 이상탐지(anomaly detection) 성능을 제공합니다. 실험을 통해 우리는 기존의 AE는 너무 큰 bottleneck을 가지면 identity function(주어진 입력을 그대로 출력하는 함수)이 되며 이상탐지 성능이 떨어지는 것에 반해, VAE는 bottleneck의 크기가 커질수록 이상탐지 성능이 오르는 효과를 갖는 것을 확인할 수 있습니다. 따라서 AE 기반의 anomaly detection을 수행할 때, 기존에는 bottleneck의 크기를 hyper-parameter로 튜닝해야 했던 반면에, VAE의 경우에는 튜닝을 할 필요가 거의 없습니다.  

- 신용 카드 데이터에 VAE(Variational Autoencoder)를 적용합니다.


- 분포를 배우는 것이 좋은 점은,  
    첫째, 데이터 및 노이즈 생성 프로세스를 명시적으로 모델링함으로써 VAE는 더 robust 하게 그 두가지를 분리하는 방법을 배울 수 있습니다.   
    둘째, disentanglement constraint가 적용되면 잠재 공간이 더 해석 가능해집니다.    
    셋째, 잠재 벡터를 샘플링하여 새 샘플을 생성하고 디코더를 통해 새로운 데이터를 생성할 수 있습니다.
    
     disentanglement : latent space가 linear한 구조를 가지게 되어서 하나의 latent vector z 를 움직였을 때 정해진 어떠한 하나의 특성이 변경되게 만들고자 하는 것. "Disentanglement constraint"는 이러한 독립적이고 해석 가능한 잠재 표현을 학습하도록 네트워크에 적용되는 제약.

### Contents

1. Variational Autoencoder 훈련
2. 잠재 표현 시각화 (Visualize Latent Representations)
3. VAE model 훈련
4. 재구성 오류 계산
5. 테스트 세트에 대한 평가

## Raw Dataset

model train 은 train set 만으로 합니다. test set은 최종 평가 때까지 사용되지 않습니다.

-  데이터 세트가 매우 불균형합니다.

In [ ]:
# 'Class' 열에 있는 각 값의 개수를 count 합니다.

In [ ]:
# 'Class' 열에서 사기(Fraud) 거래의 개수를 셉니다. 'Class'가 1인 경우가 사기 거래를 의미합니다.
# 'Class' 열에서 정상(Normal) 거래의 개수를 셉니다. 'Class'가 0인 경우가 정상 거래를 의미합니다.
# 사기 거래와 정상 거래의 전체 개수를 계산합니다.
# 전체 거래 수를 출력합니다. 콤마(,)는 천 단위 구분을 위한 것입니다.
# 사기 거래의 비율을 계산하여 퍼센트(%)로 출력합니다.
# 정상 거래의 비율을 계산하여 퍼센트(%)로 출력합니다.

- Time 필드를 일중 시간으로 변환합니다.   
- Amount 필드는 로그 스케일로 변환합니다.

- class 1 이 너무 적으므로 통상적인 7:3 대신 5:5 비율로 train, test set 분리

In [ ]:
# y_train 데이터에서 각 클래스의 비율 계산

In [ ]:
# y_test 데이터에서 각 클래스의 비율 계산

## Variational Autoencoder 훈련

- 디코더의 출력인 데이터 분포 매개 변수는 정규 분포를 따릅니다.  신용카드 데이터는 실수 값이며 일반적으로 정규 분포를 따르므로 정규 분포를 사용하여 출력을 모델링하는 것이 합리적입니다.

train set 을 정상과 비정상 set 으로 구분

In [ ]:
# y_train이 0인 데이터(정상 거래)만 선택하여 X_train_normal에 저장
# y_train이 1인 데이터(사기 거래)만 선택하여 X_train_fraud에 저장

test set을 정상과 비정상 set으로 구분

In [ ]:
# y_test가 0인 데이터(정상 거래)만 선택하여 X_test_normal에 저장
# y_test가 1인 데이터(사기 거래)만 선택하여 X_test_fraud에 저장

### 데이터 파이프라인 작성

In [ ]:
# X_train_normal 데이터셋을 텐서로 변환한 후, tf.data.Dataset 객체로 만듭니다.
# X_test_normal 데이터셋을 텐서로 변환한 후, tf.data.Dataset 객체로 만듭니다.

## 신경망 구조 정의

<img src="https://i.imgur.com/auIX6vL.png" WIDTH=600/>

### Sampling Class

먼저 'Sampling' 클래스를 빌드합니다. 이것은 인코더 출력의 평균 (mu) 및 표준 편차 (sigma)와 함께 가우스 노이즈 입력을 제공하는 맞춤형 Keras 레이어입니다. 실제로 이 레이어의 출력은 다음 방정식으로 제공됩니다.

$$z = \mu + e^{0.5\sigma} * \epsilon  $$

여기서 $\mu$ = mean, $\sigma$ = standard deviation, $\epsilon$ = random sample

In [ ]:
class Sampling(tf.keras.layers.Layer):
    def call(self, inputs):
        # 인코더의 출력을 분해합니다.
        # 배치의 크기와 차원을 얻습니다.
        # 무작위 텐서를 생성합니다.
        # 재매개변수화 기법을 적용합니다.

### Kullback–Leibler Divergence
모델의 생성 능력을 향상 시키려면 잠재 공간에 도입된 랜덤 정규 분포를 고려해야 합니다. 이를 위해 [Kullback–Leibler Divergence](https://arxiv.org/abs/2002.07514)가 계산되어 재구성 손실에 추가됩니다. 공식은 아래 함수에서 정의됩니다.

In [ ]:
def kl_reconstruction_loss(inputs, outputs, mu, sigma):
    # KLD 계산 공식 적용
    # 배치 내 평균을 계산하고 -0.5를 곱하여 최종 KLD 손실을 계산

### VAE Model
이제 전체 VAE 모델을 정의할 수 있습니다. KL reconstruction loss를 추가하기 위해 `model.add_loss()`를 사용합니다. 이 손실을 계산하는 것은 `y_true`와 `y_pred`를 사용하지 않으므로 `model.compile()`에서 사용할 수 없습니다.

- add_loss() 메서드 : 손실이 있는 경우, 자동으로 합산되어 주 손실에 추가

In [ ]:
# 잠재 공간의 차원을 2로 설정합니다. 즉, 인코더는 입력 데이터를 2차원의 잠재 벡터로 압축합니다.
# X_train_normal 데이터셋의 특징(피처) 수를 입력 데이터의 차원으로 설정합니다.

### Encoder 정의

In [ ]:
# 인코더의 입력 레이어를 정의
# 은닉층을 정의합니다.
# 잠재 변수의 평균을 나타내는 mu 레이어 정의
# 잠재 변수의 표준 편차를 나타내는 sigma 레이어 정의
# 샘플링 레이어를 통해 잠재 벡터 z를 샘플링합니다.
# 인코더 모델을 정의합니다.

### Decoder 정의

In [ ]:
# 디코더의 입력 레이어를 정의합니다.
# 은닉층을 정의
# 출력층을 정의. 원래 입력 데이터 차원으로 복원하며 출력 값을 0과 1 사이로 제한
# 디코더 모델을 정의

### 변이형 오토인코더(VAE, Variational Autoencoder)를 구축하고 학습하기 위해 필요한 모델을 정의

In [ ]:
# 인코더를 통과시켜 평균(mu), 표준 편차(sigma), 그리고 잠재 공간의 벡터(z)를 얻습니다.
# 디코더를 사용하여 잠재 공간의 벡터(z)로부터 입력을 재구성
# 입력과 재구성된 값을 사용하여 VAE 모델을 생성
# Kullback-Leibler 발산을 계산하는 함수를 사용하여 손실을 추가합니다. 이는 잠재 공간의 분포가
# 우리가 원하는 분포(예: 표준 정규 분포)에 가까워지도록 돕습니다.
# add_loss는 사용자 정의 손실을 추가하는 메서드

In [ ]:
# 학습 과정에서의 평균 손실을 추적
# 평균 제곱 오차 손실 함수를 사용합니다.

In [ ]:
def train_step(x_batch_train):
        # VAE 모델에 배치 입력을 공급하여 재구성된 결과를 얻음
        # 재구성 손실을 계산
        # KLD 정규화 손실을 추가
    # 그라디언트를 계산하고 가중치 업데이트
# 테스트 스텝 추가
def test_step(x_batch_test):
# 손실 지표를 두 개로 분리
    # 에포크 시작 시 지표 리셋
        # 훈련 스텝
        # 테스트 스텝
        # 매 500 스텝마다 결과 표시

## Reconstruction Error 시각화

- Anomaly data 와의 bar chart 비교를 위해 정상 abnormal data 갯수 만큼의 임의의 normal sample 을 선택

In [ ]:
# 정상 sample 무작위 sampling

### 정상 data 및 fraud data 의 autoencoder 재 구성값
- 정상 data의 metric

In [ ]:
# 정상 데이터의 재구성 결과 값
# 재구성 오류 -  input/output 간의 MSE 구하기

In [ ]:
# 비정상 data의 재구성 결과값
# 재구성 오류 -  input/output 간의 MSE 구하기

### MSE 분포 시각화 비교

## 잠재 표현 (Latent Representations) 시각화

In [ ]:
#  X_test의 잠재 공간에서의 평균을 산점도로 그립니다.
# 제목과 축 레이블을 설정합니다.
# 범례 추가: y_test 값에 따라 0은 Normal, 1은 Fraud로 표시합니다.
# 간단히 범례를 추가합니다.
# 그래프를 표시합니다.

원점 [0,0] 평균 벡터 주변에서 사기와 사기가 아닌 트랜잭션 사이에는 분명한 구분이 있습니다. 이는 VAE가 의미있는 것을 학습하고 있다는 신호입니다.

In [ ]:
# X_test의 잠재 표현 표준 편차 계산

빨간색으로 표시된 사기 거래의 표준 편차가 훨씬 더 흩어져 있고 사기 거래가 불규칙한 경향이 있다는 직관과 일치합니다.

In [ ]:
# X_test의 잠재 표현 시각화

샘플된 잠재 벡터 z입니다. 클러스터링 및 scattering은 이전 관찰과 일치합니다.

##  Reconstruction error threshold
- 정상 data 로만 train 한 vae model 의 재구성 결과의 원본과의 mse 차이를 기준으로 사기 거래 분류

In [ ]:
# VAE를 사용하여 train set의 데이터를 재구성(predictions)합니다.
# 원본 데이터(X_train)와 재구성된 데이터(predictions) 간의 평균 제곱 오차(MSE)를 계산
# 각 샘플에 대해 MSE를 계산하여, 재구성 오류를 측정합니다.
# 재구성 오류(mse)와 실제 클래스(y_train)를 포함한 데이터프레임을 생성
# 각 샘플의 재구성 오류와 해당 샘플이 정상(0)인지 사기(1)인지 나타내는 레이블을 포함

In [ ]:
# 'true_class' 값에 따라 정상(0)과 사기(1) 데이터로 그룹한 다음, 각 그룹의 재구성 오류에 대한 기술 통계를 구합니다.

위에서 보듯 사기성이 아닌 거래의  오류는 사기성 거래 보다 낮습니다.

- mean + 3*std를 threshold로 사용하여 test set의 재구성 error(squared error)가 threshold 보다 크면 fraud 거래로 분류합니다.  
- 1 std - 68%, 2 std - 96%, 3 std - 99.8%

In [ ]:
# 정상 클래스(true_class가 0인 데이터)의 재구성 오류에 대한 평균(mean)과 표준 편차(std)를 추출
# desc 데이터프레임에서 'reconstruction_error' 열의 평균과 표준 편차를 가져옵니다.
# 다양한 임계값(thresholds) 설정. 임계값은 3-sigma, 2.5-sigma, 2-sigma로 설정.
# 계산된 평균, 표준 편차, 그리고 임계값들을 출력

- Test set을 이용하여 재구성 오류 계산  
- thrshold 별로 confusion matrix

In [ ]:
# thresholds 딕셔너리를 value 값 기준으로 오름차순 정렬합니다.
# 모델의 예측을 수행하는 함수입니다.
# 주어진 임계값(threshold)과 MSE(Mean Squared Error)를 비교하여
# 예측 결과를 반환합니다. MSE가 임계값보다 크면 1(사기), 작으면 0(정상)으로 예측합니다.
def model_predict(threshold, mse):
# 각 sigma와 그에 대응하는 threshold 값을 기준으로 모델을 평가합니다.
    # VAE를 사용해 X_test 데이터에 대한 예측을 수행합니다.
    # MSE(Mean Squared Error)를 계산합니다.
    # 각 테스트 샘플에 대해 원본 데이터와 예측 데이터 간의 제곱 오차 평균을 계산합니다.
    # 임계값을 사용해 예측 결과를 0 또는 1로 변환합니다.
    # 정밀도(Precision)와 재현율(Recall)을 계산하고 출력합니다.
    # 혼동 행렬(Confusion Matrix)을 계산합니다.
    # 혼동 행렬을 히트맵으로 시각화합니다.